In [111]:
#region Chargement des packages
#from IPython import get_ipython;
#get_ipython().magic('reset -sf')
import os
import sys
if os.path.basename(os.getcwd())=='Prospective_conso':
    sys.path.append('../../')
from functions.f_notebook import hide_toggle

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from functions.f_tools import *
from functions.f_graphicalTools import *
from Models.Prospective_conso.f_evolution_tools import *
from mycolorpy import colorlist as mcp
import qgrid # great package https://github.com/quantopian/qgrid
import numpy as np
import time
from functools import reduce
dpe_colors = ['#009900', '#33cc33', '#B3FF00', '#e6e600', '#FFB300', '#FF4D00', '#FF0000',"#000000"]

# print(os.getcwd())
Graphic_folder = "Graphics/"
Data_folder = "data/"
# endregion

# Modélisation de l'évolution de la consommation et des émissions dans le secteur du bâtiment (chauffage résidentiel et tertiaire) en France

## Introduction <a class="anchor" id="intro"></a>

### Table of Contents

* [1. Introduction](#intro)
* [2. Hypothèses de modélisation du transport de voyageurs](#initialstate)
* [3. Hypothèses de modélisation du fret](#decarbonization)
* [4. Simulation](#simulation)
    * [4.1 Etat actuel du parc](#actuel)
    * [4.2 Résultats de la simulation prospective](#prospective)
* [5. Biblioraphie](#biblio)

## 2. Hypothèses de modélisation du parc bâti <a class="anchor" id="initialstate"></a>

In [112]:
#Four dimensions in the dataset
dim_names = ["Energy_source", "building_type", "Vecteur", "year"];
#Two main indexes
Index_names = ["Energy_source", "building_type"];
Energy_system_name = "Energy_source"
#Reading the data
data_set_from_excel = pd.read_excel(Data_folder + "Hypotheses_rester_Heating_2D.xlsx", None);
#Extracting info from sheets and creating indexes etc
sim_param = extract_sim_param(data_set_from_excel, Index_names=Index_names, dim_names=dim_names,
                              Energy_system_name=Energy_system_name)
#Creating the initial building description
sim_param["init_sim_stock"] = create_initial_parc(sim_param).sort_index()
sim_param["volume_variable_name"] = "surface"
sim_param["init_sim_stock"]["surface"] = sim_param["init_sim_stock"]["surface"] * sim_param["init_sim_stock"]["IPONDL"]

# We adjust energy need for electricity considering the primary energy factor
sim_param["init_sim_stock"]["conso_unitaire_elec"]=sim_param["init_sim_stock"]["conso_unitaire_elec"]/2.3

## for those able to make qgrid work, you should be able to modify the table sim_param['retrofit_Transition'] within the interface below
#https://github.com/quantopian/qgrid
col_def = { col :  { 'editable': True } for col in sim_param['retrofit_Transition'].columns}
qgrid_widget = qgrid.show_grid(sim_param['retrofit_Transition'], show_toolbar=True, column_definitions=col_def)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [113]:
sim_param['retrofit_Transition']=qgrid_widget.get_changed_df()

In [114]:
# When data is not given for every numeric index (typically years), we interpolate
sim_param = interpolate_sim_param(sim_param)
sim_param["retrofit_change_surface"]=sim_param["retrofit_change_total_proportion_surface"].diff().fillna(0)

# We fill the missing parameters/indexes
Para_2_fill = {param: sim_param["base_index_year"] for param in
                       ["retrofit_improvement", "retrofit_change_surface", "retrofit_Transition"]}
sim_param = complete_parameters(sim_param, Para_2_fill=Para_2_fill)
sim_param["retrofit_change_surface"]=sim_param["retrofit_change_surface"]*sim_param["init_sim_stock"]["surface"]
sim_param = complete_missing_indexes(data_set_from_excel, sim_param, Index_names, dim_names)
    
## We define some functions which will calculate at each time step the energy need, consumption, emissions...
def f_Compute_conso(x,sim_param,Vecteur):
    conso_unitaire = x["conso_unitaire_"+Vecteur]
    Energy_source = x.name[sim_param['base_index'].names.index('Energy_source')]
    seasonal_efficiency=sim_param["seasonal_efficiency"][(Energy_source,Vecteur)]
    conso_unitaire=conso_unitaire/seasonal_efficiency
    return x["energy_need_per_"+sim_param["volume_variable_name"]] * x[sim_param["volume_variable_name"]]*x["proportion_energy_need"]*conso_unitaire
for Vecteur in sim_param["Vecteurs"]:
        sim_param["f_Compute_conso_"+Vecteur]={"conso_"+Vecteur : partial(f_Compute_conso,Vecteur =Vecteur)}
        
def f_Compute_conso_totale(x,sim_param):
    res=0.
    for Vecteur in sim_param["Vecteurs"]:
        res+=x["conso_"+Vecteur]
    return res
sim_param["f_Compute_conso_totale"]={"Conso" : lambda x,sim_param: f_Compute_conso_totale(x,sim_param)}

def f_Compute_besoin(x,sim_param,Vecteur):
    conso_unitaire = x["conso_unitaire_"+Vecteur]
    return x["energy_need_per_surface"] * x["surface"]*x["proportion_energy_need"]*conso_unitaire
for Vecteur in sim_param["Vecteurs"]:
        sim_param["f_Compute_besoin_"+Vecteur]={"Besoin_"+Vecteur : partial(f_Compute_besoin,Vecteur =Vecteur)}
        
def f_Compute_besoin_total(x,sim_param):
    res=0.
    for Vecteur in sim_param["Vecteurs"]:
        res+=x["Besoin_"+Vecteur]
    return res
sim_param["f_Compute_besoin_total"]={"Besoin" : lambda x,sim_param: f_Compute_besoin_total(x,sim_param)}


def f_compute_emissions(x, sim_param, year, Vecteur):
    return sim_param["direct_emissions"].loc[Vecteur, year] * x["conso_" + Vecteur] + \
           sim_param["indirect_emissions"].loc[Vecteur, year] * x["conso_" + Vecteur]
for Vecteur in sim_param["Vecteurs"]:
        sim_param["f_Compute_emissions_"+Vecteur]={"emissions_"+Vecteur : partial(f_compute_emissions,Vecteur =Vecteur)}
    
def f_Compute_emissions_totale(x, sim_param):
    res = 0.
    for Vecteur in sim_param["Vecteurs"]:
        res += x["emissions_" + Vecteur]
    return res
sim_param["f_Compute_emissions_totale"]={"emissions" : lambda x,sim_param: f_Compute_emissions_totale(x,sim_param)}


def f_Compute_electrical_peak(x, sim_param):
    Energy_source = x.name[sim_param['base_index'].names.index('Energy_source')]
    return x["conso_elec"] * 1.5 / 35  / sim_param["peak_efficiency"][(Energy_source,"elec")]*sim_param["share_peak"][(Energy_source,"elec")]
sim_param["f_Compute_electrical_peak_totale"] = {
        "electrical_peak": lambda x, sim_param: f_Compute_electrical_peak(x, sim_param)}    

## 4.Simulation <a class="anchor" id="simulation"></a>

In [118]:
# We lanch the simulation
sim_stock = launch_simulation(sim_param)

sim_stock_df = pd.concat(sim_stock, axis=0).reset_index().\
    rename(columns={"level_0":"year"}).set_index([ "year"  ,  "Energy_source"  ,"building_type", "old_new"])

sim_param["new_yearly_surface"] should have dimensions : ['Energy_source', 'building_type', 'year'], 
 but dimensions are : ['building_type']
Error


TypeError: 'int' object is not iterable

### 4.1 Situation initiale <a class="anchor" id="actuel"></a>

In [82]:
### categories pour avoir des groupes de couleurs dans les graphiques
# col_class_dict={'Bus et cars GNV' : 1, 'Bus et cars H2':1, 'Bus et cars diesel':1, 'Bus et cars électrique':1,
#  'Rail court':2, 'Rail long':2, 'Camion fret' : 2,'camion fret H2' : 2,
#                 'VP GNV':3, 'VP fuel':3, 'VP électrique':3,
#  'VUL GNV':4, 'VUL fuel':4, 'VUL électrique':4,
#  'aerien_international':5, 'aerien_interne':5, 'aerien_outre_mer':5,
#  'deux roues diesel':6, 'deux roues électrique':6,
#  'Avion fret  international': 7, 'Bateau fret  international': 7,'Train fret':7}
# import plotly.express as px

col_class_dict={'elec' : 1, 'bois':2, 'ordures' : 2, 'autres':2, 'gaz':3, 'fioul':3,
 'charbon':3}


Var = "conso"
all_columns=[Var + "_" + Vec for Vec in sim_param['Vecteurs']]
y_df = sim_stock_df.loc[(2021,slice(None),slice(None))].reset_index().set_index(['Energy_source','building_type','old_new']).filter(all_columns)
y_df = pd.melt(y_df,value_name=Var,ignore_index=False)
y_df[['v1','Vecteur']]=y_df['variable'].str.split('_',expand=True)
y_df=y_df.drop(['v1','variable'],axis=1)
y_df=y_df.reset_index().groupby(['building_type','Vecteur']).sum().reset_index()
# y_df = y_df.loc[y_df["year"]==2021]
y_df[Var]=y_df[Var]/10**9
color_dict = gen_grouped_color_map(col_class_dict)
y_df["class"]=[col_class_dict[cat] for cat in y_df["Vecteur"]]
y_df=y_df.sort_values(by=['class'])

locals()[Var] = y_df.copy()

fig = px.bar(y_df,x="building_type", y=Var, color="Vecteur", title="Wide-Form Input",color_discrete_map=color_dict)
fig=fig.update_layout(title_text="Conso énergie finale par mode de transport (en TWh)", xaxis_title="Categorie",yaxis_title="Conso [TWh]")
fig.show()

In [83]:
Var = "emissions"
all_columns=[Var + "_" + Vec for Vec in sim_param['Vecteurs']]
y_df = sim_stock_df.loc[(2021,slice(None),slice(None))].reset_index().set_index(['Energy_source','building_type','old_new']).filter(all_columns)
y_df = pd.melt(y_df,value_name=Var,ignore_index=False)
y_df[['v1','Vecteur']]=y_df['variable'].str.split('_',expand=True)
y_df=y_df.drop(['v1','variable'],axis=1)
y_df=y_df.reset_index().groupby(['building_type','Vecteur']).sum().reset_index()
# y_df = y_df.loc[y_df["year"]==2021]
y_df[Var]=y_df[Var]/10**12
color_dict = gen_grouped_color_map(col_class_dict)
y_df["class"]=[col_class_dict[cat] for cat in y_df["Vecteur"]]
y_df=y_df.sort_values(by=['class'])

locals()[Var] = y_df.copy()

fig = px.bar(y_df,x="building_type", y=Var, color="Vecteur", title="Wide-Form Input",color_discrete_map=color_dict)
fig=fig.update_layout(title_text="Emissions de GES par vecteur énergétique (en MtCO2e)", xaxis_title="Categorie",yaxis_title="Emissions [MtCO2e]")
fig.show()

### 4.2 Résultats de la simulation prospective <a class="anchor" id="prospective"></a>

In [88]:
#region représentations graphiques

Var = "Conso"
y_df = sim_stock_df.groupby(["year",Energy_system_name])[Var].sum().to_frame().reset_index().\
    pivot(index=['year'], columns=Energy_system_name).loc[[year for year in sim_param["years"][1:]],Var]

fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Conso énergie finale par mode de transport (en TWh)", xaxis_title="Année",yaxis_title="Conso [TWh]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

# #endregion

In [91]:
Var = "emissions"
y_df = sim_stock_df.groupby(["year",Energy_system_name])[Var].sum().to_frame().reset_index().\
    pivot(index=['year'], columns=Energy_system_name).loc[[year for year in sim_param["years"][1:]],Var]/10**12

fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Emissions de GES par type de chauffage (en MtCO2e)", xaxis_title="Année",yaxis_title="Emissions [MtCO2e]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()


In [108]:
new_names={"conso_" + cat : cat for cat in sim_param["Vecteurs"]}

y_df = sim_stock_df.groupby(["year"])[[ 'conso_'+Vecteur for Vecteur in sim_param["Vecteurs"]]].sum().loc[[year for year in sim_param["years"][1:]],:].rename(columns=new_names)
y_df=y_df/10**9
fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Conso énergie finale par vecteur (en TWh)", xaxis_title="Année",yaxis_title="Conso [TWh]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

In [110]:
new_names={"emissions_" + cat : cat for cat in sim_param["Vecteurs"]}

y_df = sim_stock_df.groupby(["year"])[[ 'emissions_'+Vecteur for Vecteur in sim_param["Vecteurs"]]].sum().loc[[year for year in sim_param["years"][1:]],:].rename(columns=new_names)
y_df=y_df/10**12
fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Emissions par vecteur [MtCO2e]", xaxis_title="Année",yaxis_title="Emissions [MtCO2e]")
#plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline
fig.show()

## 5. Bibliographie <a class="biblio" id="initialstate"></a>

- [Guide très complet du gouvernement](https://www.ecologie.gouv.fr/sites/default/files/Info%20GES_Guide%20m%C3%A9thodo.pdf) donne les consommations et les émissions pour toutes les sortes de transports et les analyses qui permettent d'obtenir ces valeurs.

- [Les chiffres clés du transport](https://www.statistiques.developpement-durable.gouv.fr/edition-numerique/chiffres-cles-transports-2022/) du SDES

- [Données et études statistiques du SDES sur le transport](https://www.statistiques.developpement-durable.gouv.fr/transports)

- [Etude Prospective de France stratégie](https://www.strategie.gouv.fr/sites/strategie.gouv.fr/files/atoms/files/prospective_2040-2060_des_transports_et_des_mobilites_-_rapport_de_synthese_-_fevrier_2022.pdf) février 2022

- [Etude prospective de l'ADEME](https://librairie.ademe.fr/cadic/6531/transitions2050-rapport-compresse.pdf?modal=false) "transition 2050" le transport est p172

- [Article sur le transport maritime mondial](https://www.polytechnique-insights.com/dossiers/energie/les-innovations-bas-carbone-du-fret-maritime/comment-reduire-lempreinte-carbone-du-fret-maritime/#note-4)

- [Rapport parlementaire de 2021](http://www.senat.fr/rap/r20-604/r20-604_mono.html) Transport de marchandises face aux impératifs environnementaux

- [Un document du lobby de l'avition](https://protectiondelaerien.fr/info-ou-infox-lavion-est-il-un-moyen-de-transport-efficace/) pas inintéressant. Le problème n'est pas l'inefficacité de l'avion mais la démesure du besoin de distance/vitesse.
